# Pose Estimation with TensorFlow

## Get normalized vector

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

In [6]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Load Video
video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'
# practice1 = video_path + 'practice1.mp4'
# practice2 = video_path + 'practice2.mp4'
swingDiff_highQual = video_path + 'swingDiff_highQual.mp4'




### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,11]
]

### Vector List

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
    numberOfPeople = 2
#     lamdaVal = 0.885

#     minBPD = 10.0
#     maxBPD = 0.0
    
#     saveNum = 0
#     saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(swingDiff_highQual)    ### Change the File Here!!
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
#         keypoints_only = np.delete(keypoints_with_scores,2,2)    # remove confidence level
        keypoints_only_body = np.delete(keypoints_with_scores, [0,1,2,3,4], 1)   # raw keypoints value with confidence level
        
       # Calculate each normalized vector and average confidence score for each person
        norm_vectors_only_body = []
        for person in keypoints_only_body:
            tempPerson = []
            for i in vectorList:
                tempY = person[i[1]][0] - person[i[0]][0]
                tempX = person[i[1]][1] - person[i[0]][1]
                tempAverageS = (person[i[1]][2] + person[i[0]][2])/2
                tempVector = np.array([tempX, tempY])
                normalizedTempVector = tempVector / np.linalg.norm(tempVector)
                tempPerson.append([normalizedTempVector[0], normalizedTempVector[1], tempAverageS])
            norm_vectors_only_body.append(tempPerson)

        norm_vectors_only_body = np.array(norm_vectors_only_body)
        norm_vectors_only_body.reshape(2,12,3)

        print(norm_vectors_only_body)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

[[[0.36272594 0.2728178  0.81579566]
  [0.35089797 0.2795864  0.8163218 ]
  [0.35171628 0.2649208  0.86163497]
  [0.3622683  0.28697562 0.76991695]
  [0.363443   0.2552395  0.790252  ]
  [0.42075762 0.31050202 0.9099315 ]
  [0.42468715 0.23234816 0.91706   ]
  [0.41492766 0.3525275  0.6084575 ]
  [0.41812822 0.19297826 0.6127286 ]
  [0.40873826 0.40791512 0.8695526 ]
  [0.40488577 0.13474636 0.7854921 ]
  [0.59644955 0.29513606 0.9620515 ]
  [0.5957999  0.25192755 0.9823468 ]
  [0.73581684 0.306951   0.86976326]
  [0.7376196  0.23944925 0.95110923]
  [0.8480097  0.31249872 0.8365448 ]
  [0.8524102  0.23045981 0.8803753 ]]

 [[0.3350399  0.7545894  0.72606575]
  [0.32443598 0.76274437 0.6920335 ]
  [0.32484096 0.7463567  0.76259106]
  [0.3419658  0.7722417  0.8386316 ]
  [0.34180376 0.7366517  0.8389422 ]
  [0.40797734 0.7966213  0.9205071 ]
  [0.40258467 0.71350926 0.9407028 ]
  [0.3980229  0.8411895  0.65599614]
  [0.39492056 0.66883326 0.5781021 ]
  [0.3939578  0.90307075 0.62203115]

[[[0.36273476 0.2725431  0.810531  ]
  [0.35072267 0.2792464  0.82020664]
  [0.352063   0.26475465 0.85076714]
  [0.36190498 0.28713575 0.78085274]
  [0.36407763 0.2552018  0.7815053 ]
  [0.4217342  0.3103525  0.9059993 ]
  [0.42506886 0.2328846  0.9070693 ]
  [0.4156569  0.35272515 0.64111793]
  [0.41946858 0.19412811 0.6318734 ]
  [0.4086668  0.40758324 0.8710171 ]
  [0.40486586 0.1351128  0.81497294]
  [0.5975536  0.29517084 0.96488756]
  [0.5960381  0.25211775 0.98306686]
  [0.7364041  0.30673465 0.8598231 ]
  [0.73762894 0.23953001 0.95222425]
  [0.8484976  0.31275532 0.8024228 ]
  [0.8524801  0.23071396 0.87378   ]]

 [[0.33564198 0.75476915 0.7453249 ]
  [0.32525545 0.76292795 0.6399298 ]
  [0.32560834 0.7466544  0.76970166]
  [0.3432773  0.7725374  0.8051358 ]
  [0.34167057 0.73670304 0.83487236]
  [0.40853462 0.7958209  0.91426426]
  [0.40339768 0.7138963  0.94522685]
  [0.3978388  0.84124815 0.6350109 ]
  [0.39567578 0.6687536  0.64151406]
  [0.39338356 0.90329534 0.62472403]

[[[0.36232266 0.27247652 0.82272124]
  [0.35035568 0.27918538 0.82826215]
  [0.351651   0.26479903 0.86421096]
  [0.36159095 0.2871976  0.7942904 ]
  [0.3633934  0.2555843  0.7823742 ]
  [0.42144236 0.3123685  0.91524416]
  [0.42465657 0.23240621 0.9110381 ]
  [0.41535708 0.3533123  0.6437091 ]
  [0.41971058 0.19420013 0.61659837]
  [0.4091161  0.40731522 0.87824136]
  [0.40570018 0.13511367 0.788065  ]
  [0.5977257  0.2950739  0.9642471 ]
  [0.59659684 0.25192514 0.98279613]
  [0.73477674 0.3066807  0.8739897 ]
  [0.7380079  0.23914197 0.9475914 ]
  [0.84813106 0.31261516 0.84119344]
  [0.85331696 0.23068063 0.86975867]]

 [[0.33468172 0.7547514  0.7042952 ]
  [0.32423466 0.7627556  0.60843647]
  [0.32523403 0.7465876  0.7584263 ]
  [0.34179613 0.7722091  0.81232065]
  [0.34160802 0.73666584 0.83487976]
  [0.41008276 0.79494023 0.9168761 ]
  [0.40442848 0.71474737 0.9369434 ]
  [0.39745748 0.8422517  0.6512931 ]
  [0.39583322 0.6685875  0.68611646]
  [0.39279354 0.9037838  0.6091083 ]

In [14]:
keypoints_only_body

array([[[0.42144236, 0.3123685 , 0.91524416],
        [0.42465657, 0.23240621, 0.9110381 ],
        [0.41535708, 0.3533123 , 0.6437091 ],
        [0.41971058, 0.19420013, 0.61659837],
        [0.4091161 , 0.40731522, 0.87824136],
        [0.40570018, 0.13511367, 0.788065  ],
        [0.5977257 , 0.2950739 , 0.9642471 ],
        [0.59659684, 0.25192514, 0.98279613],
        [0.73477674, 0.3066807 , 0.8739897 ],
        [0.7380079 , 0.23914197, 0.9475914 ],
        [0.84813106, 0.31261516, 0.84119344],
        [0.85331696, 0.23068063, 0.86975867]],

       [[0.41008276, 0.79494023, 0.9168761 ],
        [0.40442848, 0.71474737, 0.9369434 ],
        [0.39745748, 0.8422517 , 0.6512931 ],
        [0.39583322, 0.6685875 , 0.68611646],
        [0.39279354, 0.9037838 , 0.6091083 ],
        [0.3878206 , 0.6075792 , 0.79264176],
        [0.59781134, 0.7782051 , 0.94313145],
        [0.59495485, 0.73143506, 0.97079396],
        [0.752001  , 0.78980106, 0.7665147 ],
        [0.75214046, 0.7141066 ,

In [34]:
norm_vectors_only_body

array([[[-0.99919307,  0.04016414,  0.91314113],
        [ 0.98913497, -0.14701028,  0.77947664],
        [-0.99172449, -0.12838434,  0.76381826],
        [ 0.99338818, -0.11480361,  0.76097524],
        [-0.97302026, -0.23071958,  0.70233166],
        [-0.09763817,  0.99522197,  0.93974566],
        [ 0.11279706,  0.99361801,  0.94691712],
        [-0.99965799, -0.02615293,  0.97352159],
        [ 0.08438762,  0.99643302,  0.9191184 ],
        [-0.09003013,  0.99593896,  0.96519375],
        [ 0.05228147,  0.99863237,  0.85759157],
        [-0.0731829 ,  0.99731851,  0.90867507]],

       [[-0.99752343, -0.07033385,  0.92690974],
        [ 0.96618974, -0.25783196,  0.78408456],
        [-0.98310179, -0.18305975,  0.81152993],
        [ 0.99713969, -0.07558015,  0.63020074],
        [-0.99148536, -0.13021824,  0.73937911],
        [-0.08879327,  0.99605012,  0.93000376],
        [ 0.08725326,  0.99618614,  0.95386869],
        [-0.9981401 , -0.06096169,  0.9569627 ],
        [ 0.074994